In [ ]:
import streamlit as st
import altair as alt
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import col, year as year_func, month as month_func, dayofmonth

session = get_active_session()

customer_id = 'CUST001004'
year = 2025
month = 1

df = session.table("PROD.SMART_METER_READINGS") \
    .filter(col("CUSTOMER_ID") == customer_id) \
    .filter(year_func(col("TIMESTAMP")) == year) \
    .filter(month_func(col("TIMESTAMP")) == month) \
    .filter(dayofmonth(col("TIMESTAMP")) <= 7) \
    .select(col("TIMESTAMP"), col("CONSUMPTION_KWH")) \
    .sort(col("TIMESTAMP"))
pd = df.to_pandas()

chart = alt.Chart(df.to_pandas()).mark_line(point=True).encode(
    x=alt.X('TIMESTAMP:T', 
            title='Date and Time',
            axis=alt.Axis(format='%b %d, %H:%M')),  # Shows: Jan 01, 14:30
    y=alt.Y('CONSUMPTION_KWH:Q', title='Energy Consumption (kWh)'),
    tooltip=[
        alt.Tooltip('TIMESTAMP:T', title='Date & Time', format='%B %d, %Y %H:%M'),
        alt.Tooltip('CONSUMPTION_KWH:Q', title='Consumption (kWh)', format='.3f')
    ]
).properties(
    title=f'Usage Trend for {customer_id} - First Week of {year}-{month:02d}',
    width=800,
    height=400
)

# Display chart in Streamlit
st.altair_chart(chart, use_container_width=True)

In [ ]:
import streamlit as st
import altair as alt
import pandas as pd
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import col, sum as sum_, avg, round as round_, count_distinct, dateadd, current_date, lit

session = get_active_session()

# Load tables
customers = session.table("PROD.CUSTOMERS")
readings = session.table("PROD.SMART_METER_READINGS")

# Join and aggregate - use DataFrame column references
df = customers.join(
    readings, 
    customers["customer_id"] == readings["CUSTOMER_ID"]  # Direct DataFrame column reference
).filter(
    readings["TIMESTAMP"] >= dateadd("day", lit(-30), current_date())
).group_by(
    customers["service_plan"], 
    customers["customer_type"]
).agg(
    count_distinct(customers["customer_id"]).alias("customer_count"),
    sum_(readings["CONSUMPTION_KWH"]).alias("total_consumption"),
    avg(readings["CONSUMPTION_KWH"]).alias("avg_consumption"),
    sum_(readings["COST_USD"]).alias("total_revenue")
).select(
    "service_plan",
    "customer_type",
    "customer_count",
    round_("total_consumption", 2).alias("total_consumption"),
    round_("avg_consumption", 3).alias("avg_consumption"),
    round_("total_revenue", 2).alias("total_revenue")
).sort(
    "total_consumption", ascending=False
)

# Simple grouped bar chart - revenue by segment
chart = alt.Chart(df.to_pandas()).mark_bar().encode(
    x=alt.X('CUSTOMER_TYPE:N', title='Customer Type'),
    y=alt.Y('TOTAL_REVENUE:Q', title='Revenue ($)', axis=alt.Axis(format='$,.0f')),
    color=alt.Color('SERVICE_PLAN:N', title='Plan'),
    xOffset='SERVICE_PLAN:N',
    tooltip=[
        'CUSTOMER_TYPE:N',
        'SERVICE_PLAN:N',
        alt.Tooltip('CUSTOMER_COUNT:Q', format=','),
        alt.Tooltip('TOTAL_REVENUE:Q', format='$,.0f')
    ]
).properties(
    width=600,
    height=400,
    title='Revenue by Customer Segment'
)


# Display chart in Streamlit
st.altair_chart(chart, use_container_width=True)